# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sid1
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sid1')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Chose sessionId along with itemInSession is my composite key as sessionId alone is not unique as a user can listen to many songs in one session and have the same sessionId for all the songs heard but along with itemInSession which is different for every song heard by the user in a particular session, the row becomes unique and is exactly what I need to run my query.

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS sid1.music_length "
query = query + """(sessionId int, itemInSession int, artist text, length float, song text, 
                     PRIMARY KEY (sessionId, itemInSession))"""
session.execute(query)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO sid1.music_length (sessionId, itemInSession, artist, length, song)"""
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (  
            int(line[8]),  # session_id
            int(line[3]),  # item_in_session
            line[0],  # artist_name
            float(line[5]),  # length
            line[9]))  # song))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
### Query 1. Give me the artist, song title and song's length in the music app history that was heard during 
### sessionId = 338, and itemInSession  = 4

query = session.execute ("""SELECT artist, song, length FROM music_length 
                            WHERE sessionId = 139 AND itemInSession  = 4""")
# Converting the ResultSet into a list of rows
rows = []
for row in query:
    rows.append((row.artist, row.song, row.length))

# Creating a Pandas DataFrame from the rows
df = pd.DataFrame(rows, columns=['artist', 'song', 'length'])
print(df)

       artist                          song      length
0  Tamba Trio  Quem Quiser Encontrar O Amor  177.188126


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

# Chose sessionId and userId as my composite key and itemInSession as my clustering column as session_id alone is not unique as a user can listen to many songs in a session which will cause the sessionId to be same for all the songs heard in a particular session, but along with userId and itemInSession as a partition which is different for each song heard in one session, the row becomes unique and is exactly what I need to run my query.

In [11]:
## Query 2
query = "CREATE TABLE IF NOT EXISTS sid1.song_playlist_session "
query = query + """(sessionId int, userId int, itemInSession int, artist text, firstName text, lastName text, song text, 
                    PRIMARY KEY ((sessionId, userId), itemInSession))"""
session.execute(query)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO sid1.song_playlist_session (sessionId, userId, itemInSession, artist, firstName, lastName, 
                   song)"""
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (            
            int(line[8]),  # session_id
            int(line[10]), # user_id
            int(line[3]),  # item_in_session
            line[0],  # artist_name
            line[1],  # first_name
            line[4],  # last_name
            line[9]))  # song

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = session.execute ("""SELECT artist, song, firstName, lastName FROM sid1.song_playlist_session 
                            WHERE sessionId = 182 AND userId = 10""")
# Appending results to a list
rows = []
for row in query:
    rows.append((row.artist, row.song, row.firstname, row.lastname))

# Creating a Pandas DataFrame
df = pd.DataFrame(rows, columns=['artist', 'song', 'firstName', 'lastName'])

# Display the DataFrame
print(df)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstName lastName  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


# Chose song and userId as my composite key as song alone is not unique and can be sung by different singers but along with userId each row unique and is exactly what I need to run my query. 

In [14]:
## Query 3
query = "CREATE TABLE IF NOT EXISTS sid1.music_history "
query = query + """(song text, userId int, firstName text, lastName text,
                    PRIMARY KEY (song, userId))"""
session.execute(query)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO sid1.music_history (song, userId, firstName, lastName)"""
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (
            line[9],  # song
            int(line[10]),   # user_id
            line[1],  # first_name
            line[4]))  # last_name

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = session.execute ("""SELECT firstName, lastName FROM music_history
                            WHERE song = 'All Hands Against His Own'""")
# Appending results to a list
rows = []
for row in query:
    rows.append((row.firstname, row.lastname))

# Creating a Pandas DataFrame
df = pd.DataFrame(rows, columns=['firstName', 'lastName'])

# Display the DataFrame
print(df)   

    firstName lastName
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [17]:
## Dropping the table before closing out the sessions

In [18]:
query = "DROP table music_length"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "DROP table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "DROP table music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()